# Code example of Symbolic Sampling for Max-Clique

In [ ]:
# make sure autoqubo is also installed in this jupyter kernel
%cd ../..
%pip install -e .
%cd examples/max_clique

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from pathlib import Path

from autoqubo import SamplingCompiler

### Helper functions to parse Max-Clique instances


In [ ]:
# make DIMACS files accessible
DATASET_PATH = "..//data/max_clique_dimacs_subset/"


instances_200 = [
    "c-fat200-1.clq",
    "c-fat200-2.clq",
    "brock200_1.clq",
    "brock200_2.clq",
    "brock200_3.clq",
    "san200_0.7_1.clq",
    "san200_0.7_2.clq",
    "san200_0.9_1.clq",
    "san200_0.9_2.clq",
    "san200_0.9_3.clq",
]
BATCH_SMALL = [Path().joinpath(DATASET_PATH + fn) for fn in instances_200]


def read_graph(file_path):
    """reads edge set from DIMACS graph
    Args:
        file_path(os.PathLike): path to dimacs formatted textfile
    Returns:
        edges(np.ndarray): adjacency matrix,  edges[v1, v2]==1 iff v1, v2 are adjacent
        vertices_num(int): number of vertices
    """
    last_vertex_has_edge = False
    with open(file_path, encoding="utf-8") as graph_file:
        for i, line in enumerate(graph_file):
            if i == 0:
                instance_name = str(Path(file_path).stem)
                print(f"Reading graph: {instance_name}")
            elif line.startswith("p"):
                _, _, vertices_num, edges_num = line.split()
                vertices_num = int(vertices_num)
                print(f"Vertices: {vertices_num}, Edges: {edges_num}")
                edges = np.zeros(shape=(vertices_num, vertices_num))
            elif line.startswith("e"):
                _, temp_1, temp_2 = line.split()
                if temp_1 == temp_2:
                    continue
                # only save one pair (i,j) per edge, i<j
                temp_1 = int(temp_1)
                temp_2 = int(temp_2)
                node_1 = min(temp_1, temp_2) - 1  # node names start with 1
                node_2 = max(temp_1, temp_2) - 1
                assert node_2 <= vertices_num, "specified number of nodes is incorrect"
                if node_2 == vertices_num - 1:
                    last_vertex_has_edge = True
                edges[node_1, node_2] = 1
            elif line.startswith("c"):  # ignore comment lines
                continue
            else:
                raise ValueError("unknown line in dimacs graph file: ", line)
    if not last_vertex_has_edge:
        raise ValueError(
            "number of vertices does not match edge set. last vertex has no edges."
        )
    return edges, vertices_num

#### Explicit Sampling

In [ ]:
# explicit-only constraint function
def old_constraint(x, edges):
    """returns number of constraint violations"""
    sum_violations = 0
    for i, j in combinations(range(len(x)), 2):
        edge = edges[i, j]
        if (x[i] == 1 and x[j] == 1) and edge == 0:
            sum_violations += 1
    return sum_violations

def cost_function(x):
    """maximize clique size"""
    return -1 * sum(x)

In [ ]:
# load example max clique instance
filename = BATCH_SMALL[0]
edges, n = read_graph(filename)

In [ ]:
# explicit constraint function defined by given graph
explicit_constraint = lambda x: old_constraint(x, edges)

In [ ]:
# obtain explicit QUBO
qubo1, offset = SamplingCompiler.generate_qubo(
    cost_function, explicit_constraint, n, penalty_method="sum", use_multiprocessing=False
)

In [ ]:
# QUBO matrix only has numeric entries
qubo1

#### Symbolic Sampling

In [ ]:
# constraint function that can handle
# both explicit and symoblic adjacency matrix
def new_constraint(x, edges):
    """returns number of constraint violations"""
    sum_violations = 0
    for i, j in combinations(range(len(x)), 2):
        edge = edges[i, j]
        if (x[i] == 1 and x[j] == 1) and edge != 1:
            sum_violations += 1 - edge
    return sum_violations

In [ ]:
# note that we could use this constraint
# the same way as before, with an explicit
# adjacency matrix
explicit_constraint = lambda x: new_constraint(x, edges)
# obtain explicit QUBO, same as above
qubo2, offset = SamplingCompiler.generate_qubo(
    cost_function, explicit_constraint, n, penalty_method="sum", use_multiprocessing=False
)
assert (qubo1 == qubo2).all()

In [ ]:
from sympy import symbols
from autoqubo.symbolic import symbolic_matrix
from autoqubo.symbolic import insert_values


# create symbolic matrix
# use example graph to determine number of symbolic variables
symbolic_edges = symbolic_matrix(edges.shape[0], edges.shape[1])
# constraint is only dependent on graph size
constraint = lambda x: new_constraint(x, symbolic_edges)

In [ ]:
# generate symbolic QUBO
symb_qubo, offset = SamplingCompiler.generate_qubo(
    cost_function, constraint, n, penalty_method="sum", use_multiprocessing=False
)

In [ ]:
# symbolic QUBO has formulas as entries
symb_qubo

In [ ]:
# insert explicit values to obtain explicit QUBO
explicit_qubo = insert_values(symb_qubo, edges)

In [ ]:
explicit_qubo

In [ ]:
# result is same as QUBO from explicit sampling
(explicit_qubo == qubo1).all()

## Runtime Comparison

In [ ]:
from timeit import default_timer as timer

DATASET = BATCH_SMALL

In [ ]:
# symbolic sampling
start_symb = timer()
# generate symbolic QUBO
symb_qubo, offset = SamplingCompiler.generate_qubo(
    cost_function, constraint, n, penalty_method="sum", use_multiprocessing=False
)
# insert explicit values to obtain explicit QUBO
for filename in DATASET:
    edges, n = read_graph(filename)
    explicit_qubo = insert_values(symb_qubo, edges)
stop_symb = timer()

In [ ]:
# explicit sampling
start_exp = timer()
# sample for each explicit QUBO
for filename in DATASET:
    edges, n = read_graph(filename)
    # explicit constraint function defined by given graph
    explicit_constraint = lambda x: old_constraint(x, edges)
    explicit_qubo, offset = SamplingCompiler.generate_qubo(
    cost_function, explicit_constraint, n, penalty_method="sum", use_multiprocessing=False
)
stop_exp = timer()

In [ ]:
symb_time = np.round(stop_symb - start_symb, 2)
exp_time = np.round(stop_exp - start_exp, 2)
print(f"Symbolic sampling took {symb_time}s in total.")
print(f"Explicit sampling took {exp_time}s in total.")
print(f"Symbolic sampling was {np.round(exp_time/symb_time, 2)}x faster for {len(DATASET)} instances.")
